In [5]:
import os

# Change working directory to your project root
os.chdir(r"C:\Users\GEN-AI\End-to-End-machine-project-with-mlflow-")

print("Changed working directory to:", os.getcwd())
print("config.yaml exists:", os.path.exists("config/config.yaml"))

Changed working directory to: C:\Users\GEN-AI\End-to-End-machine-project-with-mlflow-
config.yaml exists: True


In [6]:
import os 

In [11]:
%pwd

'C:\\Users\\GEN-AI'

In [10]:
%pwd

'C:\\Users\\GEN-AI'

In [12]:
import os

# Set working directory to your project root
os.chdir(r"C:\Users\GEN-AI\End-to-End-machine-project-with-mlflow-")

print("Current working directory:", os.getcwd())

Current working directory: C:\Users\GEN-AI\End-to-End-machine-project-with-mlflow-


In [3]:
import pandas as pd

In [13]:
data=pd.read_csv ("artifacts/data_ingestion/winequality-red.csv")
data.head()

#data.shape

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [15]:
# i want save all my schema in here and read it 
# also create all schema in dictionay
from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [16]:
# the prepre for configurtion 
from mlProject.utils.common import read_yaml,create_directories
from mlProject.constants import *

In [17]:





class ConfigurationManger:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH, # path to config file
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath) # read config
        self.params = read_yaml(params_filepath) # read params
        self.schema = read_yaml(schema_filepath) # read schema

        create_directories([self.config.artifacts_root])  # create directories if not exist


    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            
            all_schema=schema

        )
        return data_validation_config


In [18]:
# create componets 
# it willl create a list of all coumn then check each column shecma 
# # and then check if the column is in the schema or not
# if not it will raise an error 
# if yes it will return the schema of the column as status text 
import os 
from mlProject import logger


In [21]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config


    def validate_all_columns(self) -> bool:
        try:
            validation_status= None
            data= pd.read_csv(self.config.unzip_data_dir)
            all_cols =  list(data.columns)
            all_datatype = list(data.dtypes)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema and  all_datatype:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation Status :{validation_status}")
                    raise ValueError(f"Column {col} is not in the schema or datatype mismatch")
                else:
                    validation_status =True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation stats: {validation_status} for column {col} with datatype {data[col].dtype} and schema {self.config.all_schema[col]}")
            logger.info("All columns validated successfully")
            return validation_status

        except Exception as e:
            raise e


In [22]:
try:
    config = ConfigurationManger()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()  # Make sure this matches your method name
except Exception as e:
    raise e

[2025-06-07 11:43:44,038: INFO: common: YAML file config\config.yaml loaded successfully]
[2025-06-07 11:43:44,054: INFO: common: YAML file params.yaml loaded successfully]
[2025-06-07 11:43:44,065: INFO: common: YAML file schema.yaml loaded successfully]
[2025-06-07 11:43:44,071: INFO: common: Created directory at: artifacts]
[2025-06-07 11:43:44,077: INFO: common: Created directory at: artifacts/data_validation]


[2025-06-07 11:43:44,400: INFO: 135133736: All columns validated successfully]
